# Import Statements

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import yfinance as yf

from future_stock_price import predict_month_ahead
from financiallib.chartpack import FinancialPlot
from financiallib.utils import detailed_summary
from financiallib.finstats import detailed_summary as dts
from financiallib.statistical_tests import var_historic, cvar_historic

from tqdm import tqdm
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import risk_matrix
from pypfopt.efficient_frontier import EfficientFrontier
from numpy.linalg import inv
from pypfopt.hierarchical_portfolio import HRPOpt
from pypfopt import objective_functions

# Data Preparation

In [2]:
nifty_stocks = pd.read_csv("NSE550.csv")['Symbol'].apply(lambda x: x + '.NS').to_list()
downloader = False

In [3]:
try:
    os.mkdir('Data')
except:
    pass

if downloader:
    for stocks in tqdm(nifty_stocks):
        data = yf.download(stocks, start="2007-01-01", end="2023-08-01", progress=False, show_errors=False)
        data['TIC'] = stocks
        data.to_csv(f'Data/{stocks}.csv')

## Helper Functions

In [24]:
def maxDD(prices:pd.Series) -> float:
    # Drawdown
    inv = pd.Series(prices.values)

    z = pd.Series(index=range(len(inv)))
    z.iloc[0] = inv.iloc[0]

    for i in range(1, len(inv)):
        z.iloc[i] = max(inv[i], z[i-1])

    # Maximum Drawdown
    drawdowns = (inv - z)
    max_drawdown = drawdowns.min() / z.iloc[0]

    return max_drawdown


def VaR(prices:pd.Series) -> tuple:
    # var and cVar
    var = var_historic(prices.pct_change().dropna())

    return var


def highCurr(prices:pd.Series) -> float:
    # Highest and Lowest price
    high = prices.max()

    # current price
    current_price = prices.iloc[-1]
    highDist = high - current_price

    return highDist

def lowCurr(prices:pd.Series) -> float:
    # Highest and Lowest price
    low = prices.min()

    # current price
    current_price = prices.iloc[-1]
    lowDist = low - current_price

    return lowDist

## Collecting Stock Prices, Feature Calculations Etc.

In [ ]:
monthly_stocks = pd.DataFrame()

for stocks in tqdm(nifty_stocks):
    try:
        df = pd.read_csv(f"Data/{stocks}.csv", index_col=0, parse_dates=True)
        df.fillna(method='ffill', inplace=True)
        
        df['trailing_6m_returns'] = df['Adj Close'].pct_change(126)
        df['trailing_3m_returns'] = df['Adj Close'].pct_change(63)
        df['trailing_1m_returns'] = df['Adj Close'].pct_change(21)

        # df['maxDD'] = df['Adj Close'].rolling(21).apply(maxDD)
        df['VaR'] = df['Adj Close'].rolling(21).apply(VaR)
        df['hD'] = df['Adj Close'].rolling(21).apply(highCurr)
        df['lD'] = df['Adj Close'].rolling(21).apply(lowCurr)
        
        df = df.dropna()
        l1, l2, l3 = 0.16, 0.33, 0.5
        df['momentum'] = (l3 * df['trailing_1m_returns'] + l2 * df['trailing_3m_returns'] + l1 * df['trailing_6m_returns'])
        
        df = df.resample('M').last()
        
        monthly_stocks = monthly_stocks.append(df)
    except Exception as e:
        pass
    
monthly_stocks = monthly_stocks.reset_index().sort_values(['Date', 'TIC']).set_index('Date')

In [ ]:
# Drop invalid data
monthly_stocks.dropna(inplace=True)

In [ ]:
daily_prices = pd.DataFrame()

for stocks in nifty_stocks:
    df = pd.read_csv(f"Data/{stocks}.csv", index_col=0)
    df.fillna(method='ffill', inplace=True)
    
    daily_prices = daily_prices.append(df)
    
daily_prices = daily_prices.reset_index()[['Date', 'Adj Close', 'TIC']].set_index(['Date', 'TIC']).unstack()
daily_prices.columns = daily_prices.columns.droplevel()
daily_prices.columns.name = None

In [ ]:
# Filling missing values
daily_prices.fillna(method="ffill", inplace=True)
daily_prices.fillna(method="bfill", inplace=True)

In [ ]:
monthly_prices = pd.DataFrame()

for stocks in tqdm(nifty_stocks):
    try:
        df = pd.read_csv(f"Data/{stocks}.csv", index_col=0, parse_dates=True)
        df.fillna(method='ffill', inplace=True)
        
        df = df.resample('M').last()
        monthly_prices = monthly_prices.append(df)
    except:
        pass

100%|██████████| 2144/2144 [01:07<00:00, 31.54it/s]


In [ ]:
# fill missing values
monthly_prices = monthly_prices.sort_index()
monthly_prices = monthly_prices.fillna(method="ffill")
monthly_prices.fillna(method="bfill", inplace=True)

monthly_prices = monthly_prices.drop_duplicates().reset_index()[['Date', 'Adj Close', 'TIC']].set_index(['Date', 'TIC']).unstack()
monthly_prices.columns = monthly_prices.columns.droplevel()
monthly_prices.columns.name = None

monthly_prices = monthly_prices.dropna(how="all", axis=1)
monthly_prices.fillna(method="ffill", inplace=True)

# Utility Functions

In [ ]:
def backtest(cash, backtest_df, weights):
    amount_allocation = {}
    shares, total, balance, total_invested = {}, 0, 0, 0
    prices = backtest_df.iloc[0].to_dict()
    new_prices = backtest_df.iloc[len(backtest_df)-1].to_dict()

    for keys in backtest_df.columns:
        amount_allocation[keys] = weights[keys] * cash

    for keys in backtest_df.columns:
        shares[keys] = (amount_allocation[keys] // prices[keys])

    for keys in backtest_df.columns:
        total_invested = total_invested + (shares[keys] * prices[keys])

    balance = cash - total_invested

    for keys in backtest_df.columns:
        total = total + (shares[keys] * new_prices[keys])

    return total_invested, total, balance, shares


def backtest_lastm(cash, backtest_df, weights):
    amount_allocation = {}
    shares, total, balance, total_invested = {}, 0, 0, 0
    prices = backtest_df.to_dict()

    for keys in backtest_df.index:
        amount_allocation[keys] = weights[keys] * cash

    for keys in backtest_df.index:
        shares[keys] = (amount_allocation[keys] // prices[keys])

    for keys in backtest_df.index:
        total_invested = total_invested + (shares[keys] * prices[keys])

    balance = cash - total_invested

    return total_invested, np.nan, balance, shares

# Backtesting

In [ ]:
monthly_stocks.index = pd.Series(monthly_stocks.index).apply(lambda x: x.strftime("%Y-%m-%d"))
dates_to_trade = monthly_stocks.index.unique()

dates_to_trade = dates_to_trade[dates_to_trade > "2017-12-31"]

In [19]:
N = 10
cash = 75000
flag = 0

investment, got_back, balances, date, all_weights = [], [], [], [], []
turnover_stack = []
turnover = []
opt_method = "min_vol"


for dates in range(len(dates_to_trade)):
    
    # choose stocks to invest
    temp = monthly_stocks.loc[dates_to_trade[dates]].sort_values(['momentum'], ascending=[False])[:N]
    turnover.append(len(set(temp['TIC'].to_list()).difference(set(turnover_stack))) / N)
    turnover_stack = temp['TIC'].to_list()
    
    # backtest
    if dates != len(dates_to_trade) - 1:
        new_df = monthly_prices.loc[dates_to_trade[dates]:dates_to_trade[dates+1], temp['TIC'].to_list()]
    else:
        new_df = monthly_prices.loc[dates_to_trade[dates], temp['TIC'].to_list()]
        flag = 1 
        
    if opt_method != "EQ":
        returns = {}
        for tickers in temp['TIC'].to_list():
            try:
                returns[tickers] = predict_month_ahead(ticker=tickers, future_date=dates_to_trade[dates]).values[0]
            except ValueError:
                returns[tickers] = monthly_prices.loc[:dates_to_trade[dates], tickers][-4:].pct_change().dropna().mean() * 12

        returns = pd.Series(returns)
    
    
    # Max Sharpe Optimisation and EQ, Min Vol
    if opt_method == "EQ":
        weights = {s:1/N for s in temp['TIC'].to_list()}
        
    elif opt_method.lower() == "max_sharpe":
        cov = risk_matrix(daily_prices.loc[:dates_to_trade[dates], temp['TIC'].to_list()][-60:], method="ledoit_wolf")
        
        ef = EfficientFrontier(returns, cov)
        
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
        )
    
    elif opt_method.lower() == "min_vol":
        cov = risk_matrix(daily_prices.loc[:dates_to_trade[dates], temp['TIC'].to_list()][-60:], method="ledoit_wolf")
        
        ef = EfficientFrontier(returns, cov, verbose=False)
        ef.add_objective(objective_functions.L2_reg, gamma=1)

        ef.min_volatility()
        weights = ef.clean_weights()
        
    elif (opt_method.lower()=="kelly"):
        cov = risk_matrix(daily_prices.loc[:dates_to_trade[dates], temp['TIC'].to_list()][-60:], method="ledoit_wolf")
        precision_matrix = pd.DataFrame(inv(cov), index=temp['TIC'].to_list(), columns=temp['TIC'].to_list()) 
        
        kelly_wt = precision_matrix.dot(returns).clip(lower=0).values
        kelly_wt /= np.sum(np.abs(kelly_wt))
        wts = dict(zip(temp['TIC'].to_list(), kelly_wt))

        weights = wts
        
    elif (opt_method.upper()=="HRP"):
        returns = monthly_prices.loc[:dates_to_trade[dates], temp['TIC'].to_list()].pct_change().dropna() * 12
        hrp = HRPOpt(returns)
        wts = hrp.optimize()
        weights = wts
    
    
    all_weights.append(weights)
    
    # Last month check
    if flag == 0:
        invested, total, balance, _ = backtest(cash, new_df, weights)
    else:
        invested, total, balance, _ = backtest_lastm(cash, new_df, weights)
        
    date.append(dates_to_trade[dates])
    investment.append(invested)
    got_back.append(total)
    balances.append(balance)
    
    # Recalculate cash at next month end
    cash = total + balance

In [20]:
df_accounts = pd.DataFrame({"date": date, "invested": investment, "value_next_month": got_back, "balance": balances, "turnover":turnover})
df_accounts['total_portfolio'] = df_accounts['invested'] + df_accounts['balance'] 
df_accounts['returns'] = np.log(1 + df_accounts['invested'].pct_change())

# Final Results

In [21]:
df_accounts.head()

,date,invested,value_next_month,balance,turnover,total_portfolio,returns
0,2018-01-31,997563.271343,811054.601650,2436.728657,1.0,1000000.000000,NaN
1,2018-02-28,812264.243090,736183.389471,1227.087217,0.4,813491.330307,-0.205490
2,2018-03-31,735793.990429,645602.206363,1616.486260,0.6,737410.476688,-0.098876
3,2018-04-30,645208.899251,604106.140564,2009.793371,0.9,647218.692622,-0.131376
4,2018-05-31,605098.027304,539188.086593,1017.906631,0.8,606115.933935,-0.064184


In [22]:
baseline = yf.download("^CNX100", start="2007-01-01", progress=False).resample("M").last().loc[df_accounts.iloc[0, 0]:df_accounts.iloc[-1, 0]][['Adj Close']]
baseline.index = pd.Series(baseline.index).apply(lambda x: x.strftime("%Y-%m-%d"))
baseline["returns"] = baseline['Adj Close'].pct_change()
baseline = baseline.reset_index().rename(columns={'Date':'date'})

In [23]:
weights_hist = pd.concat([df_accounts['date'], pd.DataFrame(all_weights)], axis=1)

with pd.ExcelWriter(f'Results/{opt_method}_wt_mom5.xlsx') as writer:
    df_accounts.to_excel(writer, sheet_name='Portfolio', index=False)
    baseline.to_excel(writer, sheet_name='Baseline', index=False)
    weights_hist.to_excel(writer, sheet_name='Weights', index=False)

In [26]:
dts(excelSheet=f"Results/{opt_method}_wt_mom5.xlsx", sheets=["Portfolio", "Baseline"], period=12, prev_yr="2017")